In [1]:
import pandas as pd
import re
from datetime import datetime

In [2]:
earlier_games = pd.read_csv("early_career_games.csv")
later_games = pd.read_csv("late_career_games.csv")

games = pd.concat([earlier_games, later_games])

games.head()

,Unnamed: 0.1,Unnamed: 0,Players,Result,Accuracy,Moves,Date,Unnamed: 6,game_link
0,0,10 min,zainsiddiqii (557) methburger682 (356),1 0,42.3 57.6,62,"Dec 6, 2020",NaN,https://www.chess.com/game/live/5900312365
1,1,10 min,JimenaGallardo (438) zainsiddiqii (643),0 1,77.7 64.7,40,"Dec 9, 2020",NaN,https://www.chess.com/game/live/5922717305
2,2,10 min,DanielBrown1974 (577) zainsiddiqii (531),1 0,76.4 58.3,25,"Dec 9, 2020",NaN,https://www.chess.com/game/live/5923131073
3,3,10 min,zainsiddiqii (625) RRR0007 (615),1 0,81.4 77.5,64,"Dec 9, 2020",NaN,https://www.chess.com/game/live/5923181350
4,4,10 min,zainsiddiqii (564) prune131 (639),0 1,31.6 55.5,12,"Dec 10, 2020",NaN,https://www.chess.com/game/live/5927374018


In [3]:
games.isna().sum()

Unnamed: 0.1       0
Unnamed: 0         0
Players            0
Result             0
Accuracy          45
Moves              0
Date               0
Unnamed: 6      9900
game_link          0
dtype: int64

In [4]:
games.dtypes

Unnamed: 0.1      int64
Unnamed: 0       object
Players          object
Result           object
Accuracy         object
Moves             int64
Date             object
Unnamed: 6      float64
game_link        object
dtype: object

In [5]:
games.drop(columns = ["Unnamed: 0.1", "Unnamed: 6"], inplace = True)
games.rename(columns = {"Unnamed: 0": "time_control"}, inplace = True)
games.columns = games.columns.str.lower()
for col in games.columns:
    if games[col].dtype == 'object':
        games[col].str.strip()
games.head()

,time_control,players,result,accuracy,moves,date,game_link
0,10 min,zainsiddiqii (557) methburger682 (356),1 0,42.3 57.6,62,"Dec 6, 2020",https://www.chess.com/game/live/5900312365
1,10 min,JimenaGallardo (438) zainsiddiqii (643),0 1,77.7 64.7,40,"Dec 9, 2020",https://www.chess.com/game/live/5922717305
2,10 min,DanielBrown1974 (577) zainsiddiqii (531),1 0,76.4 58.3,25,"Dec 9, 2020",https://www.chess.com/game/live/5923131073
3,10 min,zainsiddiqii (625) RRR0007 (615),1 0,81.4 77.5,64,"Dec 9, 2020",https://www.chess.com/game/live/5923181350
4,10 min,zainsiddiqii (564) prune131 (639),0 1,31.6 55.5,12,"Dec 10, 2020",https://www.chess.com/game/live/5927374018


In [6]:
print(games.time_control.value_counts())

time_control
5 min     4418
10 min    2788
3 min     2550
5|5         81
1 min       23
3|2         20
30 min       7
2|1          6
15|10        5
20 min       2
Name: count, dtype: int64


In [7]:
games.time_control.replace(["5 min", "10 min", "3 min", "1 min", "30 min", "20 min"],
                           ["5|0", "10|0", "3|0", "1|0", "30|0", "20|0"],
                           inplace = True)
games.time_control = games.time_control.str.replace("|", "+")
games.time_control.value_counts()

time_control
5+0      4418
10+0     2788
3+0      2550
5+5        81
1+0        23
3+2        20
30+0        7
2+1         6
15+10       5
20+0        2
Name: count, dtype: int64

In [8]:
games["time_format"] = games.time_control.apply(lambda x: "Rapid" if len(x.split("+")[0]) == 2
                                                else ("Blitz" if (int(x[0]) >= 3) & (int(x[0]) <=5)
                                                else "Bullet"))
games.time_format.value_counts()

time_format
Blitz     7069
Rapid     2802
Bullet      29
Name: count, dtype: int64

In [9]:
games["colour"] = games.players.apply(lambda x: "White" if x.split(" ")[0] == "zainsiddiqii" else "Black")
games.colour.value_counts()

colour
White    4968
Black    4932
Name: count, dtype: int64

In [10]:
games["my_rating"] = games.apply(lambda x: x["players"].split()[1] if x["colour"] == "White"
                                 else x["players"].split()[-1],
                                 axis = 1).str.replace("(", "").str.replace(")", "")

games["opponent_rating"] = games.apply(lambda x: x["players"].split()[-1] if x["colour"] == "White"
                                       else (x["players"].split()[2] if (x["players"].split()[0] == "WFM") & (x["colour"] == "Black")
                                       else x["players"].split()[1]),
                                       axis = 1).str.replace("(", "").str.replace(")", "")

games["opponent"] = games.apply(lambda x: x["players"].split()[0] if x["colour"] == "Black"
                                else x["players"].split()[2],
                                axis = 1)

games.my_rating = games.my_rating.astype("int")
games.opponent_rating = games.opponent_rating.astype("int")
games.head()

# print(games[games.players.str.contains("Klokan64")])

,time_control,players,result,accuracy,moves,date,game_link,time_format,colour,my_rating,opponent_rating,opponent
0,10+0,zainsiddiqii (557) methburger682 (356),1 0,42.3 57.6,62,"Dec 6, 2020",https://www.chess.com/game/live/5900312365,Rapid,White,557,356,methburger682
1,10+0,JimenaGallardo (438) zainsiddiqii (643),0 1,77.7 64.7,40,"Dec 9, 2020",https://www.chess.com/game/live/5922717305,Rapid,Black,643,438,JimenaGallardo
2,10+0,DanielBrown1974 (577) zainsiddiqii (531),1 0,76.4 58.3,25,"Dec 9, 2020",https://www.chess.com/game/live/5923131073,Rapid,Black,531,577,DanielBrown1974
3,10+0,zainsiddiqii (625) RRR0007 (615),1 0,81.4 77.5,64,"Dec 9, 2020",https://www.chess.com/game/live/5923181350,Rapid,White,625,615,RRR0007
4,10+0,zainsiddiqii (564) prune131 (639),0 1,31.6 55.5,12,"Dec 10, 2020",https://www.chess.com/game/live/5927374018,Rapid,White,564,639,prune131


In [11]:
games["outcome"] = games.apply(lambda x: "W" if (x["colour"] == "White" and x["result"][0] == "1")
                           or (x["colour"] == "Black" and x["result"][-1] == "1") else
                              ("L" if (x["colour"] == "White" and x["result"][0] == "0")
                           or (x["colour"] == "Black" and x["result"][-1] == "0") else "D"),
                           axis = 1)

print(games.outcome.value_counts())
games.head()

outcome
W    4878
L    4583
D     439
Name: count, dtype: int64


,time_control,players,result,accuracy,moves,date,game_link,time_format,colour,my_rating,opponent_rating,opponent,outcome
0,10+0,zainsiddiqii (557) methburger682 (356),1 0,42.3 57.6,62,"Dec 6, 2020",https://www.chess.com/game/live/5900312365,Rapid,White,557,356,methburger682,W
1,10+0,JimenaGallardo (438) zainsiddiqii (643),0 1,77.7 64.7,40,"Dec 9, 2020",https://www.chess.com/game/live/5922717305,Rapid,Black,643,438,JimenaGallardo,W
2,10+0,DanielBrown1974 (577) zainsiddiqii (531),1 0,76.4 58.3,25,"Dec 9, 2020",https://www.chess.com/game/live/5923131073,Rapid,Black,531,577,DanielBrown1974,L
3,10+0,zainsiddiqii (625) RRR0007 (615),1 0,81.4 77.5,64,"Dec 9, 2020",https://www.chess.com/game/live/5923181350,Rapid,White,625,615,RRR0007,W
4,10+0,zainsiddiqii (564) prune131 (639),0 1,31.6 55.5,12,"Dec 10, 2020",https://www.chess.com/game/live/5927374018,Rapid,White,564,639,prune131,L


In [12]:
games.date = pd.to_datetime(games.date, format='%b %d, %Y')
games.head()

,time_control,players,result,accuracy,moves,date,game_link,time_format,colour,my_rating,opponent_rating,opponent,outcome
0,10+0,zainsiddiqii (557) methburger682 (356),1 0,42.3 57.6,62,2020-12-06,https://www.chess.com/game/live/5900312365,Rapid,White,557,356,methburger682,W
1,10+0,JimenaGallardo (438) zainsiddiqii (643),0 1,77.7 64.7,40,2020-12-09,https://www.chess.com/game/live/5922717305,Rapid,Black,643,438,JimenaGallardo,W
2,10+0,DanielBrown1974 (577) zainsiddiqii (531),1 0,76.4 58.3,25,2020-12-09,https://www.chess.com/game/live/5923131073,Rapid,Black,531,577,DanielBrown1974,L
3,10+0,zainsiddiqii (625) RRR0007 (615),1 0,81.4 77.5,64,2020-12-09,https://www.chess.com/game/live/5923181350,Rapid,White,625,615,RRR0007,W
4,10+0,zainsiddiqii (564) prune131 (639),0 1,31.6 55.5,12,2020-12-10,https://www.chess.com/game/live/5927374018,Rapid,White,564,639,prune131,L


In [13]:
print(games[games.accuracy == "0  0"])

     time_control                                       players result   
231          10+0        u2002994  (1737)  zainsiddiqii  (1703)   1  0  \
233          10+0  zainsiddiqii  (1703)  madhan_chauhan  (1663)   1  0   
234          10+0  madhan_chauhan  (1670)  zainsiddiqii  (1696)   0  1   
317          10+0        Tank1111  (1552)  zainsiddiqii  (1579)   1  0   
4878          5+0        zainsiddiqii  (1353)  ch3sskju  (1312)   1  0   

     accuracy  moves       date                                    game_link   
231      0  0     32 2023-03-22  https://www.chess.com/game/live/75805353065  \
233      0  0     40 2023-03-21  https://www.chess.com/game/live/75804712275   
234      0  0     26 2023-03-21  https://www.chess.com/game/live/75747117743   
317      0  0     41 2023-03-13  https://www.chess.com/game/live/76090195579   
4878     0  0     41 2021-12-13  https://www.chess.com/game/live/75828739293   

     time_format colour  my_rating  opponent_rating        opponent outcom

In [14]:
games.accuracy = games.accuracy.astype("string")
print(games.accuracy.value_counts(dropna = False))
games.dropna(inplace=True)
games = games[(games.accuracy != "Review") &
              (games.accuracy != "0  0")]

games["my_accuracy"] = games.apply(lambda x: x["accuracy"].split()[0] if x["colour"] == "White"
                                   else x["accuracy"].split(" ")[-1],
                                   axis = 1).str.strip()

games["opponent_accuracy"] = games.apply(lambda x: x["accuracy"].split()[0] if x["colour"] == "Black"
                                         else x["accuracy"].split()[-1],
                                         axis = 1).str.strip()

games.my_accuracy = games.my_accuracy.astype("float")
games.opponent_accuracy = games.opponent_accuracy.astype("float")
games.info()

accuracy
Review        245
<NA>           45
100  100        7
72.3  78.8      5
0  0            5
             ... 
72.4  85.1      1
66.5  66.3      1
75.6  84        1
58.8  84.7      1
74  96.8        1
Name: count, Length: 9270, dtype: Int64
<class 'pandas.core.frame.DataFrame'>
Index: 9605 entries, 0 to 4949
Data columns (total 15 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   time_control       9605 non-null   object        
 1   players            9605 non-null   object        
 2   result             9605 non-null   object        
 3   accuracy           9605 non-null   string        
 4   moves              9605 non-null   int64         
 5   date               9605 non-null   datetime64[ns]
 6   game_link          9605 non-null   object        
 7   time_format        9605 non-null   object        
 8   colour             9605 non-null   object        
 9   my_rating          9605 non-null   int32 

In [15]:
games.head()

,time_control,players,result,accuracy,moves,date,game_link,time_format,colour,my_rating,opponent_rating,opponent,outcome,my_accuracy,opponent_accuracy
0,10+0,zainsiddiqii (557) methburger682 (356),1 0,42.3 57.6,62,2020-12-06,https://www.chess.com/game/live/5900312365,Rapid,White,557,356,methburger682,W,42.3,57.6
1,10+0,JimenaGallardo (438) zainsiddiqii (643),0 1,77.7 64.7,40,2020-12-09,https://www.chess.com/game/live/5922717305,Rapid,Black,643,438,JimenaGallardo,W,64.7,77.7
2,10+0,DanielBrown1974 (577) zainsiddiqii (531),1 0,76.4 58.3,25,2020-12-09,https://www.chess.com/game/live/5923131073,Rapid,Black,531,577,DanielBrown1974,L,58.3,76.4
3,10+0,zainsiddiqii (625) RRR0007 (615),1 0,81.4 77.5,64,2020-12-09,https://www.chess.com/game/live/5923181350,Rapid,White,625,615,RRR0007,W,81.4,77.5
4,10+0,zainsiddiqii (564) prune131 (639),0 1,31.6 55.5,12,2020-12-10,https://www.chess.com/game/live/5927374018,Rapid,White,564,639,prune131,L,31.6,55.5


In [20]:
games = games.sort_values(by = ["date"])
games.drop(columns = ['players','result','accuracy','game_link'], inplace = True)
games.head()

,time_control,moves,date,time_format,colour,my_rating,opponent_rating,opponent,outcome,my_accuracy,opponent_accuracy
0,10+0,62,2020-12-06,Rapid,White,557,356,methburger682,W,42.3,57.6
1,10+0,40,2020-12-09,Rapid,Black,643,438,JimenaGallardo,W,64.7,77.7
2,10+0,25,2020-12-09,Rapid,Black,531,577,DanielBrown1974,L,58.3,76.4
3,10+0,64,2020-12-09,Rapid,White,625,615,RRR0007,W,81.4,77.5
4,10+0,12,2020-12-10,Rapid,White,564,639,prune131,L,31.6,55.5


In [22]:
games.to_csv('\data\games_cleaned.csv', index = False)